# SoftOrdering1DCNN on Santander Customer Transaction Prediction (SCTP)

In this notebook we will apply the neural network architecture winner of the 2nd place on MoA (https://www.kaggle.com/c/lish-moa/discussion/202256) to the Santander customer transaction prediction problem.

The results of this network (which we call SoftOrdering1DCNN) are benchmarked against a plain MLP and LightGBM.

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

import torch
from torch import nn
from torch.utils.data import DataLoader,TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [3]:
# needed for deterministic output
SEED = 2
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

# device in which the model will be trained
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

***
## data preparation

In [4]:
dataset = pd.read_csv("../input/santander-customer-transaction-prediction/train.csv")
dataset

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,...,6.1415,13.2305,3.9901,0.9388,18.0249,-1.7939,2.1661,8.5326,16.6660,-17.8661
199996,train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,...,4.9611,4.6549,0.6998,1.8341,22.2717,1.7337,-2.1651,6.7419,15.9054,0.3388
199997,train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,...,4.0651,5.4414,3.1032,4.8793,23.5311,-1.5736,1.2832,8.7155,13.8329,4.1995
199998,train_199998,0,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,...,2.6840,8.6587,2.7337,11.1178,20.4158,-0.0786,6.7980,10.0342,15.5289,-13.9001


In [5]:
# dataset split: train 60% - valid 20% - test 20%

index = np.array(dataset.index)
np.random.shuffle(index)
n = len(index)

train_index = index[0:int(0.6*n)]
valid_index = index[int(0.6*n):int(0.8*n)]
test_index = index[int(0.8*n):]

train_dset = dataset.loc[train_index].reset_index(drop=True)
valid_dset = dataset.loc[valid_index].reset_index(drop=True)
test_dset = dataset.loc[test_index].reset_index(drop=True)

In [6]:
input_features = dataset.columns[2:].tolist()
target = "target"

In [8]:
# parsing inputs as pytorch tensor dataset

train_tensor_dset = TensorDataset(
    torch.tensor(train_dset[input_features].values, dtype=torch.float),
    torch.tensor(train_dset[target].values.reshape(-1,1), dtype=torch.float)
)

valid_tensor_dset = TensorDataset(
    torch.tensor(valid_dset[input_features].values, dtype=torch.float),
    torch.tensor(valid_dset[target].values.reshape(-1,1), dtype=torch.float)
)

test_tensor_dset = TensorDataset(
    torch.tensor(test_dset[input_features].values, dtype=torch.float),
    torch.tensor(test_dset[target].values.reshape(-1,1), dtype=torch.float) 
)

***
## 3-layers MLP

In [9]:
class DNN(pl.LightningModule):

    def __init__(self, input_dim, output_dim, nn_depth, nn_width, dropout, momentum):
        super().__init__()

        self.bn_in = nn.BatchNorm1d(input_dim, momentum=momentum)
        self.dp_in = nn.Dropout(dropout)
        self.ln_in = nn.Linear(input_dim, nn_width, bias=False)

        self.bnorms = nn.ModuleList([nn.BatchNorm1d(nn_width, momentum=momentum) for i in range(nn_depth-1)])
        self.dropouts = nn.ModuleList([nn.Dropout(dropout) for i in range(nn_depth-1)])
        self.linears = nn.ModuleList([nn.Linear(nn_width, nn_width, bias=False) for i in range(nn_depth-1)])
        
        self.bn_out = nn.BatchNorm1d(nn_width, momentum=momentum)
        self.dp_out = nn.Dropout(dropout/2)
        self.ln_out = nn.Linear(nn_width, output_dim, bias=False)

        self.loss = nn.BCEWithLogitsLoss()

    def forward(self, x):
        x = self.bn_in(x)
        x = self.dp_in(x)
        x = nn.functional.relu(self.ln_in(x))

        for bn_layer,dp_layer,ln_layer in zip(self.bnorms,self.dropouts,self.linears):
            x = bn_layer(x)
            x = dp_layer(x)
            x = ln_layer(x)
            x = nn.functional.relu(x)
            
        x = self.bn_out(x)
        x = self.dp_out(x)
        x = self.ln_out(x)
        return x

    def training_step(self, batch, batch_idx):
        X, y = batch
        y_hat = self.forward(X)
        loss = self.loss(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        X, y = batch
        y_hat = self.forward(X)
        loss = self.loss(y_hat, y)
        self.log('valid_loss', loss)
        
    def test_step(self, batch, batch_idx):
        X, y = batch
        y_logit = self.forward(X)
        y_probs = torch.sigmoid(y_logit).detach().cpu().numpy()
        loss = self.loss(y_logit, y)
        metric = roc_auc_score(y.cpu().numpy(), y_probs)
        self.log('test_loss', loss)
        self.log('test_metric', metric)
        
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=1e-2, momentum=0.9)
        scheduler = {
            'scheduler': ReduceLROnPlateau(
                optimizer, 
                mode="min", 
                factor=0.5, 
                patience=5, 
                min_lr=1e-5),
            'interval': 'epoch',
            'frequency': 1,
            'reduce_on_plateau': True,
            'monitor': 'valid_loss',
        }
        return [optimizer], [scheduler]

In [10]:
model = DNN(
    input_dim=len(input_features), 
    output_dim=1, 
    nn_depth=3, 
    nn_width=256, 
    dropout=0.2, 
    momentum=0.1
)

early_stop_callback = EarlyStopping(
   monitor='valid_loss',
   min_delta=.0,
   patience=20,
   verbose=True,
   mode='min'
)

trainer = pl.Trainer(callbacks=[early_stop_callback], min_epochs=10, max_epochs=200, gpus=1)

In [11]:
model.summarize()

  | Name     | Type              | Params
-----------------------------------------------
0 | bn_in    | BatchNorm1d       | 400   
1 | dp_in    | Dropout           | 0     
2 | ln_in    | Linear            | 51.2 K
3 | bnorms   | ModuleList        | 1.0 K 
4 | dropouts | ModuleList        | 0     
5 | linears  | ModuleList        | 131 K 
6 | bn_out   | BatchNorm1d       | 512   
7 | dp_out   | Dropout           | 0     
8 | ln_out   | Linear            | 256   
9 | loss     | BCEWithLogitsLoss | 0     
-----------------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)

In [12]:
trainer.fit(
    model, 
    DataLoader(train_tensor_dset, batch_size=2048, shuffle=True, num_workers=4),
    DataLoader(valid_tensor_dset, batch_size=2048, shuffle=False, num_workers=4)
)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [13]:
# AUC on validation dataset
trainer.test(model, DataLoader(valid_tensor_dset, batch_size=2048, shuffle=False, num_workers=4))

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.25559571385383606, 'test_metric': 0.8592893481254578}
--------------------------------------------------------------------------------


[{'test_loss': 0.25559571385383606, 'test_metric': 0.8592893481254578}]

In [14]:
# AUC on test dataset
trainer.test(model, DataLoader(test_tensor_dset, batch_size=2048, shuffle=False, num_workers=4))

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.25388413667678833, 'test_metric': 0.8598867058753967}
--------------------------------------------------------------------------------


[{'test_loss': 0.25388413667678833, 'test_metric': 0.8598867058753967}]

***
## SoftOrdering1DCNN

In [15]:
class SoftOrdering1DCNN(pl.LightningModule):

    def __init__(self, input_dim, output_dim, sign_size=32, cha_input=16, cha_hidden=32, 
                 K=2, dropout_input=0.2, dropout_hidden=0.2, dropout_output=0.2):
        super().__init__()

        hidden_size = sign_size*cha_input
        sign_size1 = sign_size
        sign_size2 = sign_size//2
        output_size = (sign_size//4) * cha_hidden

        self.hidden_size = hidden_size
        self.cha_input = cha_input
        self.cha_hidden = cha_hidden
        self.K = K
        self.sign_size1 = sign_size1
        self.sign_size2 = sign_size2
        self.output_size = output_size
        self.dropout_input = dropout_input
        self.dropout_hidden = dropout_hidden
        self.dropout_output = dropout_output

        self.batch_norm1 = nn.BatchNorm1d(input_dim)
        self.dropout1 = nn.Dropout(dropout_input)
        dense1 = nn.Linear(input_dim, hidden_size, bias=False)
        self.dense1 = nn.utils.weight_norm(dense1)

        # 1st conv layer
        self.batch_norm_c1 = nn.BatchNorm1d(cha_input)
        conv1 = conv1 = nn.Conv1d(
            cha_input, 
            cha_input*K, 
            kernel_size=5, 
            stride = 1, 
            padding=2,  
            groups=cha_input, 
            bias=False)
        self.conv1 = nn.utils.weight_norm(conv1, dim=None)

        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = sign_size2)

        # 2nd conv layer
        self.batch_norm_c2 = nn.BatchNorm1d(cha_input*K)
        self.dropout_c2 = nn.Dropout(dropout_hidden)
        conv2 = nn.Conv1d(
            cha_input*K, 
            cha_hidden, 
            kernel_size=3, 
            stride=1, 
            padding=1, 
            bias=False)
        self.conv2 = nn.utils.weight_norm(conv2, dim=None)

        # 3rd conv layer
        self.batch_norm_c3 = nn.BatchNorm1d(cha_hidden)
        self.dropout_c3 = nn.Dropout(dropout_hidden)
        conv3 = nn.Conv1d(
            cha_hidden, 
            cha_hidden, 
            kernel_size=3, 
            stride=1, 
            padding=1, 
            bias=False)
        self.conv3 = nn.utils.weight_norm(conv3, dim=None)
        

        # 4th conv layer
        self.batch_norm_c4 = nn.BatchNorm1d(cha_hidden)
        conv4 = nn.Conv1d(
            cha_hidden, 
            cha_hidden, 
            kernel_size=5, 
            stride=1, 
            padding=2, 
            groups=cha_hidden, 
            bias=False)
        self.conv4 = nn.utils.weight_norm(conv4, dim=None)

        self.avg_po_c4 = nn.AvgPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm2 = nn.BatchNorm1d(output_size)
        self.dropout2 = nn.Dropout(dropout_output)
        dense2 = nn.Linear(output_size, output_dim, bias=False)
        self.dense2 = nn.utils.weight_norm(dense2)

        self.loss = nn.BCEWithLogitsLoss()

    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = nn.functional.celu(self.dense1(x))

        x = x.reshape(x.shape[0], self.cha_input, self.sign_size1)

        x = self.batch_norm_c1(x)
        x = nn.functional.relu(self.conv1(x))

        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = nn.functional.relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c3(x)
        x = self.dropout_c3(x)
        x = nn.functional.relu(self.conv3(x))

        x = self.batch_norm_c4(x)
        x = self.conv4(x)
        x =  x + x_s
        x = nn.functional.relu(x)

        x = self.avg_po_c4(x)

        x = self.flt(x)

        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = self.dense2(x)

        return x

    def training_step(self, batch, batch_idx):
        X, y = batch
        y_hat = self.forward(X)
        loss = self.loss(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        X, y = batch
        y_hat = self.forward(X)
        loss = self.loss(y_hat, y)
        self.log('valid_loss', loss)
        
    def test_step(self, batch, batch_idx):
        X, y = batch
        y_logit = self.forward(X)
        y_probs = torch.sigmoid(y_logit).detach().cpu().numpy()
        loss = self.loss(y_logit, y)
        metric = roc_auc_score(y.cpu().numpy(), y_probs)
        self.log('test_loss', loss)
        self.log('test_metric', metric)
        
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=1e-2, momentum=0.9)
        scheduler = {
            'scheduler': ReduceLROnPlateau(
                optimizer, 
                mode="min", 
                factor=0.5, 
                patience=5, 
                min_lr=1e-5),
            'interval': 'epoch',
            'frequency': 1,
            'reduce_on_plateau': True,
            'monitor': 'valid_loss',
        }
        return [optimizer], [scheduler]

In [16]:
model = SoftOrdering1DCNN(
    input_dim=len(input_features), 
    output_dim=1, 
    sign_size=16, 
    cha_input=64, 
    cha_hidden=64, 
    K=2, 
    dropout_input=0.3, 
    dropout_hidden=0.3, 
    dropout_output=0.2
)

early_stop_callback = EarlyStopping(
   monitor='valid_loss',
   min_delta=.0,
   patience=21,
   verbose=True,
   mode='min'
)

trainer = pl.Trainer(callbacks=[early_stop_callback], min_epochs=10, max_epochs=200, gpus=1)

In [17]:
model.summarize()

   | Name          | Type              | Params
-----------------------------------------------------
0  | batch_norm1   | BatchNorm1d       | 400   
1  | dropout1      | Dropout           | 0     
2  | dense1        | Linear            | 205 K 
3  | batch_norm_c1 | BatchNorm1d       | 128   
4  | conv1         | Conv1d            | 641   
5  | ave_po_c1     | AdaptiveAvgPool1d | 0     
6  | batch_norm_c2 | BatchNorm1d       | 256   
7  | dropout_c2    | Dropout           | 0     
8  | conv2         | Conv1d            | 24.6 K
9  | batch_norm_c3 | BatchNorm1d       | 128   
10 | dropout_c3    | Dropout           | 0     
11 | conv3         | Conv1d            | 12.3 K
12 | batch_norm_c4 | BatchNorm1d       | 128   
13 | conv4         | Conv1d            | 321   
14 | avg_po_c4     | AvgPool1d         | 0     
15 | flt           | Flatten           | 0     
16 | batch_norm2   | BatchNorm1d       | 512   
17 | dropout2      | Dropout           | 0     
18 | dense2        | Linear       

In [18]:
trainer.fit(
    model, 
    DataLoader(train_tensor_dset, batch_size=2048, shuffle=True, num_workers=4),
    DataLoader(valid_tensor_dset, batch_size=2048, shuffle=False, num_workers=4)
)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [19]:
# AUC on validation dataset
trainer.test(model, DataLoader(valid_tensor_dset, batch_size=2048, shuffle=False, num_workers=4))

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.2868420481681824, 'test_metric': 0.8670674562454224}
--------------------------------------------------------------------------------


[{'test_loss': 0.2868420481681824, 'test_metric': 0.8670674562454224}]

In [20]:
# AUC on test dataset
trainer.test(model, DataLoader(test_tensor_dset, batch_size=2048, shuffle=False, num_workers=4))

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.28403544425964355, 'test_metric': 0.8681577444076538}
--------------------------------------------------------------------------------


[{'test_loss': 0.28403544425964355, 'test_metric': 0.8681577444076538}]

***
## LightGBM

In [ ]:
train_dataset = lgb.Dataset(
    train_dset[input_features].values,
    train_dset[target].values,
    free_raw_data=False
)

valid_dataset = lgb.Dataset(
    valid_dset[input_features].values,
    valid_dset[target].values,
    free_raw_data=False
)

In [ ]:
model_params = dict(
    objective = "binary",
    learning_rate = 0.1,
    num_leaves = 32,
    seed = 2,
    deterministic = True,
    metric = "auc"
)

In [ ]:
model = lgb.train(
    model_params, 
    train_dataset, 
    valid_sets=[valid_dataset,],
    num_boost_round=1000,
    early_stopping_rounds=20,
    verbose_eval=50,
)

In [ ]:
# AUC on validation dataset
model.best_score["valid_0"]["auc"]

In [ ]:
# AUC on test dataset
preds = model.predict(test_dset[input_features].values)
roc_auc_score(test_dset[target].values, preds)

***